In [90]:
import pandas as pd 
import numpy as np
from pyproj import Proj

In [102]:
safety_df = pd.read_csv('Safetyspd.csv')

In [103]:
from haversine import haversine, Unit

## Examples 
# Coordinates (latitude, longitude)
loc1 = (47.62207249, -122.346939293)  # Paris
loc2 = (47.613342, -122.32301)  # London
# Calculate distance in meters
distance = haversine(loc1, loc2, unit=Unit.MILES)

print(distance)

1.2672609816453408


In [110]:
df = pd.read_csv('data/df_all_listResults.csv')

In [111]:
def cal_harversine(loc1, loc2_list):
    res_list = []
    for loc2 in loc2_list:
        dist = haversine(loc1, loc2, unit=Unit.MILES)
        res_list.append(dist)
    
    return res_list

In [112]:
crime_locs = df.apply(lambda x: (x['latitude'], x['longitude']), axis=1)
df['crime_dists'] = list(map(lambda x : cal_harversine(x , crime_locs), zip(df['latitude'], df['longitude'])))

In [113]:
from scipy import stats
b = 2
df['crime_cnt_2b'] = list(map(lambda x : sum([1 if i < 0.035*b else 0 for i in x]), df['crime_dists']))
df['safety_pr'] = list(map(lambda x :100 - stats.percentileofscore(df['crime_cnt_2b'], x), df['crime_cnt_2b']))
df = df.drop('crime_dists', axis = 1)

In [114]:
df.to_csv('data/df_all_listResults_w_crime.csv', index=False)